In [2]:
import pandas as pd
import numpy as np 
import pickle 
import os
import openslide
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Lambda
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.backend import clear_session
from tensorflow.keras.backend import bias_add, constant



#Batch Preprocessing 


def preprocessing_fn(input_batch, network_input_patch_width = 1000):
    org_input_size = tf.shape(input_batch)[1]

    scaled_input_batch = tf.cast(input_batch, 'float') / 255.

    resized_input_batch = tf.cond(tf.equal(org_input_size, network_input_patch_width),
                                  lambda: scaled_input_batch,
                                  lambda: tf.image.resize(scaled_input_batch,
                                                          (network_input_patch_width, network_input_patch_width)))

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    data_format = "channels_last"
    mean_tensor = constant(-np.array(mean))

    standardized_input_batch = bias_add(resized_input_batch, mean_tensor, data_format)
    standardized_input_batch /= std

    return standardized_input_batch



#load DenseNet with KimiaNet's weights
#available at:
#https://kimialab.uwaterloo.ca/kimia/index.php/sdm_downloads/kimianet-weights/

weights_address = '\\kimianet weights\\KimiaNetKerasWeights_final.h5'
dnx = DenseNet121(include_top=False, weights=weights_address, input_shape=(1000, 1000, 3), pooling='avg')
kn_feature_extractor = Model(inputs=dnx.input, outputs=GlobalAveragePooling2D()(dnx.layers[-3].output))

kn_feature_extractor_seq = Sequential([Lambda(preprocessing_fn,
                                                  arguments={'network_input_patch_width': 1000},
                                                  input_shape=(None, None, 3), dtype=tf.uint8)])

kn_feature_extractor_seq.add(kn_feature_extractor)

## This is a function that returns a patch by coordinates obtained from 5x magnification and returns the patch at 20x mag. 

In [3]:
slide_file = "slide_file.svs"
slide = openslide.open_slide(slide_file)


def get_a_patch(slide, x, y, step1=1, step2=1):
    x = int(x)
    y = int(y)
    mag = int(slide.properties['openslide.objective-power'])

    if (mag == 20):
        x = x * 8
        y = y * 8
        
        step1 = int(step1 * 1000)
        step2 = int(step2 * 1000)
        
        img = slide.read_region((x, y), level=0, size=(step1, step2)).convert('RGB')

    else:
        x = x * 16
        y = y * 16
        
        step1 = int(step1 * 2000)
        step2 = int(step2 * 2000)

        img = slide.read_region((x, y), level=0, size=(step1, step2)).convert('RGB').resize((int(step1/2), int(step2/2)))

    return img

## Extract deep features from tissue patches of size 1000x1000

In [ ]:
import skimage.io
import pickle

#read patches and save them first
#here we load saved patches and get deep features
image_path = "\\image_path"
os.chdir(image_path)
image_files = os.listdir()
images = np.array([skimage.io.imread(x) for x in image_files])

dfs = []

for i in range(len(image_files)):

    img = images[i]
    f = kn_feature_extractor_seq.predict(tf.expand_dims(img, axis = 0))
    dfs.append(f)
    
    
with open("dfs.pkl", "wb") as f:
    pickle.dump(conv_dfs, f)